In [26]:
import sys
import types

let's create a plain function

In [1]:
def add(a, b):
    return a+b

let's take a look at its properties

In [2]:
hasattr(add, "__get__")

True

In [3]:
hasattr(add, "__set__")

False

#### that means that functions are non-data descriptors

-------------------------------

arguments for `__get__`: 

- self (instance of the descriptor - function `add`),

- instance (in this case is None, 'cause function is called from the module), 

- owner class (our main module again)

---------------------------------------

let's get a handle to main module

In [6]:
main_module = sys.modules['__main__']

In [7]:
main_module

<module '__main__'>

let's call `__get__` method

In [10]:
f = add.__get__(None, main_module)
f

<function __main__.add(a, b)>

In [11]:
f is add

True

____________________________

In [12]:
class Person:
    def __init__(self, name):
        self.name = name
    def say_hello(self):
        return f"{self.name} says hello!"

let's access function say_hello from the class

In [13]:
Person.say_hello # __get__ returns a function object - same as above

<function __main__.Person.say_hello(self)>

access it from an instance

In [15]:
p = Person("Alec")

In [16]:
p.say_hello # __get__ returns a bound method

<bound method Person.say_hello of <__main__.Person object at 0x0000025C8BC50788>>

we can also get an access to it this way:

In [17]:
bound_method = Person.say_hello.__get__(p, Person)
bound_method

<bound method Person.say_hello of <__main__.Person object at 0x0000025C8BC50788>>

`__get__` method in the function returns new bound method object every time being accessed

In [18]:
f1=p.say_hello
f2=p.say_hello

In [19]:
f1, f2

(<bound method Person.say_hello of <__main__.Person object at 0x0000025C8BC50788>>,
 <bound method Person.say_hello of <__main__.Person object at 0x0000025C8BC50788>>)

In [20]:
f1 is f2

False

In [21]:
p.say_hello()

'Alec says hello'

In [22]:
bound_method()

'Alec says hello'

In [23]:
type(bound_method)

method

where does it store its reference to the say_hello function?

In [24]:
bound_method.__func__

<function __main__.Person.say_hello(self)>

In [25]:
p.say_hello.__func__

<function __main__.Person.say_hello(self)>

that's how methods keep track of which function they are actually bound to

--------------------


let's mimic that behavior

In [ ]:
class Person:
    def __init__(self, name):
        self.name = name
    def say_hello(self):
        return f"{self.name} says hello!"
    
# we want to write a descriptor to replace say_hello

In [30]:
# define at module level
def say_hello(self):
        return f"{self.name} says hello!"

In [27]:
help(types.MethodType)

Help on class method in module builtins:

class method(object)
 |  method(function, instance)
 |  
 |  Create a bound instance method object.
 |  
 |  Methods defined here:
 |  
 |  __call__(self, /, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __delattr__(self, name, /)
 |      Implement delattr(self, name).
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __get__(self, instance, owner, /)
 |      Return an attribute of instance, which is of type owner.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __reduce__(...)
 |      Helper for pickle.
 |  
 |  __repr__(self, /)

In [28]:
class Person:
    def __init__(self, name):
        self.name = name

In [29]:
p = Person("Alec")

In [31]:
m = types.MethodType(say_hello, p)

In [32]:
m

<bound method say_hello of <__main__.Person object at 0x0000025C8BC5E848>>

In [33]:
p

In [34]:
m.__func__

<function __main__.say_hello(self)>

In [35]:
m()

'Alec says hello!'

let's implement descriptor

In [36]:
class MyFunc:
    def __init__(self, func):
        self._func = func
        
    def __get__(self, instance, owner_class):
        if instance is None:
            print("__get__ was called from class")
            return self._func
        else:
            print("__get__ was called from instance")
            return types.MethodType(self._func, instance)

In [37]:
def hello(self):
    print(f"{self.name} says hello!")

In [38]:
class Person:
    def __init__(self, name):
        self.name = name
        
    say_hello =  MyFunc(hello)


In [39]:
Person.say_hello

__get__ was called from class


<function __main__.hello(self)>

In [40]:
p = Person("Alec")

In [41]:
p.say_hello

__get__ was called from instance


<bound method hello of <__main__.Person object at 0x0000025C8BBA9A08>>

In [42]:
p.say_hello.__func__

__get__ was called from instance


<function __main__.hello(self)>

In [43]:
p.say_hello()

__get__ was called from instance
Alec says hello!
